In [1]:
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers import Activation
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
import numpy as np
import re
from collections import defaultdict

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
import gensim
import math
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.word2vec import Word2Vec 

Using TensorFlow backend.


In [2]:
LabeledSentence = gensim.models.doc2vec.LabeledSentence 
regex = "([@][A-Za-z0-9]+)|([^0-9A-Za-z# \t])|(\w+:\/\/\S+)|(#[^A-Za-z0-9]+)"
tweets = []
labels = []

In [3]:
with open('../../../data/train_semeval2018task2/tweets_us.text', 'r') as file:
    for tweet in file:
        reg_tweet = ' '.join(re.sub(regex, " ", tweet).split())
        low_tweet = reg_tweet.lower()
        tweets.append(low_tweet)
with open('../../../data/train_semeval2018task2/tweets_us.labels', 'r') as file:
    for label in file:
        tweet_label = int(label)
        labels.append(tweet_label)

In [4]:
# max_length = math.ceil(sum([len(s.split(" ")) for s in tweets])/len(tweets))
max_length = 15
print(max_length)

15


In [5]:
tokenizer = Tokenizer(filters='!"$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', split=" ",lower=True)
tokenizer.fit_on_texts(tweets)
encoded_docs = tokenizer.texts_to_sequences(tweets)
print(len(encoded_docs), "encoded doc len")

(488553, 'encoded doc len')


In [6]:
Xtrain = pad_sequences(encoded_docs[0:300000], maxlen=max_length,padding='post')

Ytrain = []

In [7]:
for emoji in labels[0:300000]:
    num = int(emoji)
    bit_vec = np.zeros(20)
    bit_vec[num] = 1
    Ytrain.append(bit_vec)

Ytrain = np.asarray(Ytrain)

In [8]:
print(len(Xtrain), len(Xtrain[0]), "one")
print(len(Ytrain), Ytrain[0].shape, Ytrain[0:5], labels[0:5], "two")

(300000, 15, 'one')
(300000, (20,), array([[ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.]]), [2, 17, 0, 18, 1], 'two')


In [9]:
Xval = pad_sequences(encoded_docs[300000:400000], maxlen=max_length,padding='post')

Yval = []#np.asarray([one_hot(emoji, 20) for emoji in labels[-200:]])

for emoji in labels[300000:400000]:
    num = int(emoji)
    bit_vec = np.zeros(20)
    bit_vec[num] = 1
    Yval.append(bit_vec)

Yval = np.asarray(Yval)

In [10]:
print len(Xval)
print len(Yval)

100000
100000


In [11]:
Xtest = pad_sequences(encoded_docs[400000:488553], maxlen=max_length,padding='post')

Ytest = []#np.asarray([one_hot(emoji, 20) for emoji in labels[-200:]])

for emoji in labels[400000:488553]:
    num = int(emoji)
    bit_vec = np.zeros(20)
    bit_vec[num] = 1
    Ytest.append(bit_vec)

Ytest = np.asarray(Ytest)

In [12]:
print(len(Xtest), Xtest[0] ,len(Xtest[0]), "three")
print(len(Ytest), Ytest[0:5], "four")
print(encoded_docs[0:5])

(88553, array([  271,   296,    24,   677,    11,     3,   365,   258,    32,
          25,    17,   247,    44,  3684, 19771], dtype=int32), 15, 'three')
(88553, array([[ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.]]), 'four')
[[153, 211, 6591, 68], [289, 61, 5, 64, 2488, 16, 1, 647, 2990, 29014], [1518, 40, 7, 3834, 5872, 207, 349], [2489, 9011, 80], [3, 130, 925, 11439, 9]]


In [14]:
vocab = tokenizer.word_index
raw_embedding = KeyedVectors.load_word2vec_format('../model_swm_300-6-10-low.w2v', binary=False)

In [15]:
def get_map_from_file(path):
    map_path = path
    input_map = defaultdict(list)
    with open(map_path, 'rb') as f:
        lines = f.read().splitlines()
        for l in lines[1:]:
            splits = l.decode('utf-8').split('\t')
            input_map[splits[0]] = [float(num) for num in splits[1:]]
    return input_map

In [16]:
emotion_map = get_map_from_file("../../../data/NRC-emotion-lexicon-wordlevel-v0.92_new.txt")
hash_map = get_map_from_file("../../../data/NRC-Hashtag-Emotion-Lexicon-v0.2_new_new.txt")

In [17]:
# print emotion_map['love']
# print hash_map['love']
# print raw_embedding['love']
arr = np.concatenate([emotion_map['love'], hash_map['love'],raw_embedding['love']])
print arr.shape

(320,)


In [18]:
vocab_size = len(vocab) + 1
weight_matrix = np.zeros((vocab_size, 320))
print(weight_matrix[17])
for word, i in vocab.items():
    #if word in raw_embedding2:
    #    weight_matrix[i] = raw_embedding2[word]
    emotion_val = np.zeros(10)
    hash_val = np.zeros(10)
    embedding = np.zeros(300)
    if word in emotion_map:
        emotion_val = emotion_map[word]
    if word in hash_map:
        hash_val = hash_map[word]
    if word in raw_embedding:
        embedding = raw_embedding[word]
    final = np.concatenate([embedding, emotion_val, hash_val])
    weight_matrix[i] = final
print(weight_matrix[17])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

In [19]:
embedding_layer = Embedding(vocab_size, 320, weights=[weight_matrix], input_length=max_length, trainable=False,
                            mask_zero=True)

In [20]:
model = Sequential()
model.add(embedding_layer)
model.add(Bidirectional(LSTM(128, dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.2)))
model.add(Dense(20, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="weights.hdf5", verbose=1, save_best_only=True)

In [21]:
hist = model.fit(Xtrain, 
                 Ytrain,
                 epochs=3,
                 batch_size=64,
                 validation_data=(Xval, Yval), 
                 callbacks=[checkpointer])

Train on 300000 samples, validate on 100000 samples
Epoch 1/4
300000/300000 [==============================] - 1153s 4ms/step - loss: 2.2422 - acc: 0.3358 - val_loss: 2.1682 - val_acc: 0.3535
Epoch 2/4
300000/300000 [==============================] - 1193s 4ms/step - loss: 2.1453 - acc: 0.3595 - val_loss: 2.1394 - val_acc: 0.3614
Epoch 3/4
300000/300000 [==============================] - 1110s 4ms/step - loss: 2.0983 - acc: 0.3703 - val_loss: 2.1304 - val_acc: 0.3621
Epoch 4/4
300000/300000 [==============================] - 1059s 4ms/step - loss: 2.0640 - acc: 0.3787 - val_loss: 2.1315 - val_acc: 0.3629


In [22]:
model.load_weights('weights.hdf5')
predicted = model.predict(Xtest)

In [23]:
score, acc = model.evaluate(Xtest, Ytest)

88553/88553 [==============================] - 123s 1ms/step


In [24]:
print(score)
print(acc)

2.12229707361
0.365001750365
